In [ ]:
# # Imports

from arcgis.gis import GIS
gis = GIS("home")

from arcgis.features import FeatureLayer

# Parameters

# National Weather Service "CurrentWarnings" layer; includes Tornadoes and other severe/extreme events
# Nix that - using the version from Esri over the original agency layer ONCE AGAIN...because Esri just does it better.
tornado_live_url = r"https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/NWS_Watches_Warnings_v1/FeatureServer/8"
tornado_live_layer = FeatureLayer(tornado_live_url)

# Layer in which to copy historical tornado warnings
tornado_archive_layer = gis.content.get("86b302d398774ea6a90809dc8a264f93").layers[0]

# Query to grab only tornado warnings from the above layer
# If I ever want more than one warning, use IN, e.g.: query = "Event IN ('Tornado Warning', 'Flash Flood Warning')"
query = "Event = 'Tornado Warning'"

# Trying to remember why I used both .to_dict() and .features...
tornado_live_features = tornado_live_layer.query(where=query).features

# # Update NWS Watches Warnings (Historical) Layer
if tornado_live_features:

    # List of Uids from NWS Watches and Warnings (historical hosted)
    tornado_archive_ids = [i.attributes["Uid"] for i in tornado_archive_layer.query().features]

    # Construct list of features that are NOT already in NWS Hist layer
    tornado_features_new = [n for n in tornado_live_features if not n.attributes["Uid"] in tornado_archive_ids]

    print(f"Tornado Warnings in current live layer: {len(tornado_live_features)}")
    print(f"Tornado Warnings not already in Historical: {len(tornado_features_new)}")

    # Cram the trimmed list of Features into NWS Historical layer
    if tornado_features_new:
        update_tornado_archive = tornado_archive_layer.edit_features(adds=tornado_features_new)
    else:
        update_tornado_archive = "No features to add to NWS Historical layer"

    # Print NWS Watches Warnings that have been added to Hist layer
    update_tornado_archive

########## ########## ########## ########## ########## ##########

# # Delete all rows in Impacted Historical and NWS Older than 100 Days
# tornado_archive_layer.delete_features(where="End_ <= CURRENT_TIMESTAMP - 365")

print("Script completed successfully")


Tornado Warnings in current live layer: 1
Tornado Warnings not already in Historical: 1
